In [3]:
import tensorflow as tf
import numpy as np

In [9]:
x_data = [[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3, 4]
          , [4, 1, 5, 5], [1, 7, 5, 5], [1, 2, 5, 6]
          , [1, 6, 6, 6], [1, 7, 7, 7]]
y_data = [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0]
          , [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]]

X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
       
    # Testing & One-hot encoding
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.arg_max(a, 1)))
    

    all = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9], 
                                          [1, 3, 4, 3], 
                                          [1, 1, 0, 1]]})
    print(all, sess.run(tf.arg_max(all, 1)))

0 1.66879
200 0.612362
400 0.500261
600 0.405152
800 0.313262
1000 0.238918
1200 0.215502
1400 0.196574
1600 0.180614
1800 0.166977
2000 0.155197
[[  3.20392335e-03   9.96785522e-01   1.05946019e-05]] [1]
[[  3.20392335e-03   9.96785522e-01   1.05946019e-05]
 [  8.90025139e-01   9.69969779e-02   1.29778404e-02]
 [  1.55670250e-08   3.58181307e-04   9.99641776e-01]] [1 0 2]


# Predict animals type
## One-hot

If the input indices is rank N, the output will have rank N+1.   
The new axis is created at dimension axis (default: the new axis is appended at the end). https://www.tensorflow.org/api_docs/python/tf/one_hot 

In [10]:
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6, shape=(?, 1)
Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot shape=(?, 1, 7)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) # shape=(?, 7)

In [12]:
# Predicting animal type based on various features
xy = np.loadtxt('data04.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6

Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print(Y.shape)  # (?, 1)
print(Y_one_hot.shape)  # (?, 1, 7)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print(Y_one_hot.shape)  # (?, 7)

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,

(?, 1)
(?, 1, 7)
(?, 7)


In [13]:
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)  ############################################

correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={
                                X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
               step, loss, acc))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

Step:     0	Loss: 5.269	Acc: 7.92%
Step:   100	Loss: 0.619	Acc: 87.13%
Step:   200	Loss: 0.396	Acc: 89.11%
Step:   300	Loss: 0.295	Acc: 93.07%
Step:   400	Loss: 0.236	Acc: 96.04%
Step:   500	Loss: 0.198	Acc: 96.04%
Step:   600	Loss: 0.170	Acc: 96.04%
Step:   700	Loss: 0.149	Acc: 96.04%
Step:   800	Loss: 0.132	Acc: 99.01%
Step:   900	Loss: 0.119	Acc: 99.01%
Step:  1000	Loss: 0.107	Acc: 99.01%
Step:  1100	Loss: 0.098	Acc: 99.01%
Step:  1200	Loss: 0.090	Acc: 99.01%
Step:  1300	Loss: 0.084	Acc: 99.01%
Step:  1400	Loss: 0.078	Acc: 100.00%
Step:  1500	Loss: 0.073	Acc: 100.00%
Step:  1600	Loss: 0.069	Acc: 100.00%
Step:  1700	Loss: 0.065	Acc: 100.00%
Step:  1800	Loss: 0.061	Acc: 100.00%
Step:  1900	Loss: 0.058	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 True Y